### Import modules

In [1]:
import torch as t
import torch.nn as nn #For neural networks

### Defining a custom neural network

In [2]:
class custom_nn (nn.Module):
    def __init__(self, n_inputs, n_hidden_neurons, n_outputs):
        super(custom_nn, self).__init__()
        self.model = nn.Sequential(nn.Linear(n_inputs, n_hidden_neurons),
                                   nn.ReLU(),
                                   nn.Linear(n_hidden_neurons, n_outputs),
                                   nn.Tanh()
                                   )
    
    def forward(self, x):
        return self.model(x)

### Training a neural network

In [7]:
%%time
ninputs = 10 #No of inputs
nhidden = 5 #No of hidden neurons
noutputs = 10 #No of outputs
nexamples = 15 #No of examples to given to the neural network

x = t.randn(nexamples, ninputs)
y = x #Output is same as input - an autoencoder
print(x.shape)
print(y.shape)
  
nnCPU = custom_nn(ninputs, nhidden, noutputs) #Create an object (nn1) of custom_nn class

loss_fn = nn.MSELoss() #Loss function
optimizer = t.optim.SGD(nnCPU.parameters(), lr = 0.01) #Defining the optimizer - SGD
epochs = 100000

print("Wait - training the model")
#Gradient Descent Algorithm
for i in range(epochs):
    ypred = nnCPU(x) #Calculate predicted value of the model
    loss = loss_fn(ypred, y) #Apply the loss function (MSE) to calculate MSE
    if ((i+1) % 10000 == 0):
        print("Epoch : ", i+1, "Loss = ", loss.item()) #Prints loss after each 10,000 epochs
    loss.backward() #Backward - Backward propagation
    optimizer.step() #Update all parameters
    optimizer.zero_grad() #Set gradients of all parameters to zero before starting the next epoch
    
print("Loss = ", loss.item()) #Prints the final loss

torch.Size([15, 10])
torch.Size([15, 10])
Wait - training the model
Epoch :  10000 Loss =  0.231475368142128
Epoch :  20000 Loss =  0.2016017884016037
Epoch :  30000 Loss =  0.19060766696929932
Epoch :  40000 Loss =  0.18365898728370667
Epoch :  50000 Loss =  0.17885242402553558
Epoch :  60000 Loss =  0.17546415328979492
Epoch :  70000 Loss =  0.17297407984733582
Epoch :  80000 Loss =  0.17093497514724731
Epoch :  90000 Loss =  0.1693805754184723
Epoch :  100000 Loss =  0.16825567185878754
Loss =  0.16825567185878754
CPU times: user 2min 39s, sys: 3.03 s, total: 2min 42s
Wall time: 27 s


### Predicting output for unseen inputs

In [8]:
x_new = t.randn(1, ninputs) #New inputs
y_new = nnCPU(x_new) #Predicting outputs using the trained model
print("x_new = ", x_new)
print("y_new = ", y_new) 

x_new =  tensor([[-0.4134, -0.5242, -0.8779, -0.2582,  0.1580,  0.8885, -1.2871, -0.5490,
          0.2534,  0.1095]])
y_new =  tensor([[-0.9356,  0.0761, -1.0000,  0.9911,  0.8794, -0.8312, -0.9758, -0.9239,
          0.6623, -0.0621]], grad_fn=<TanhBackward>)


### Training the neural network and data on a GPU if it is available

In [9]:
%%time

if t.cuda.is_available():
    device = t.device("cuda:0") #If a GPU is availble, set device as first GPU (0 means first GPU)
    print("Running on the GPU = ", device)
else:
    device = t.device("cpu") #Else set the device as CPU
    print("Running on the CPU = ", device)

ninputs = 10 #No of inputs
nhidden = 5 #No of hidden neurons
noutputs = 10 #No of outputs
nexamples = 15 #No of examples to given to the neural network

x = t.randn(nexamples, ninputs, device=device)   #IMPORTANT - x is now on GPU if a GPU is available
y = x #Output is same as input - an autoencoder  #IMPORTANT - y too is now on GPU if a GPU is available

#IMPORTANT - be cautious if data is too large and if GPU memory is not fit enough for the data 

print(x.shape)
print(y.shape)
  
nnGPU = custom_nn(ninputs, nhidden, noutputs) #Create an object (nn1) of custom_nn class
nnGPU = nnGPU.to(device) #IMPORTANT - Neural network nn1 is passed to GPU if a GPU is available

loss_fn = nn.MSELoss() #Loss function
optimizer = t.optim.SGD(nnGPU.parameters(), lr = 0.01) #Defining the optimizer - SGD
epochs = 100000

print("Wait - training the model")
#Gradient Descent Algorithm
for i in range(epochs):
    ypred = nnGPU(x) #Calculate predicted value of the model
    loss = loss_fn(ypred, y) #Apply the loss function (MSE) to calculate MSE
    if ((i+1) % 10000 == 0):
        print("Epoch : ", i+1, "Loss = ", loss.item()) #Prints loss after each 10,000 epochs
    loss.backward() #Backward - Backward propagation
    optimizer.step() #Update all parameters
    optimizer.zero_grad() #Set gradients of all parameters to zero before starting the next epoch
    
print("Loss = ", loss.item()) #Prints the final loss

Running on the GPU =  cuda:0
torch.Size([15, 10])
torch.Size([15, 10])
Wait - training the model
Epoch :  10000 Loss =  0.23078830540180206
Epoch :  20000 Loss =  0.2047799676656723
Epoch :  30000 Loss =  0.19573847949504852
Epoch :  40000 Loss =  0.19138577580451965
Epoch :  50000 Loss =  0.1889788806438446
Epoch :  60000 Loss =  0.18722932040691376
Epoch :  70000 Loss =  0.18587522208690643
Epoch :  80000 Loss =  0.18514207005500793
Epoch :  90000 Loss =  0.18463115394115448
Epoch :  100000 Loss =  0.18409566581249237
Loss =  0.18409566581249237
CPU times: user 50.9 s, sys: 1.36 s, total: 52.2 s
Wall time: 52.5 s


### Predicting output for unseen inputs

In [11]:
x_new = t.randn(1, ninputs) #New inputs
nnGPU = nnGPU.to("cpu") #IMPORTANT - nnGPU transfered back to the CPU as x_new is on CPU
y_new = nnGPU(x_new) #Predicting outputs using the trained model
print("x_new = ", x_new)
print("y_new = ", y_new) 

x_new =  tensor([[-1.4502,  0.7624,  0.0699, -0.2669,  1.1746,  1.1821,  0.9249,  0.1458,
          0.2466, -0.8550]])
y_new =  tensor([[-0.9316,  0.9683, -0.4776, -0.9994,  0.9971, -0.9475, -0.0310,  0.4689,
          0.9862, -1.0000]], grad_fn=<TanhBackward>)
